In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [4]:
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

train=train.drop(['id'],axis=1)
#test=test.drop(['id'],axis=1)

In [9]:
train.columns[train.dtypes=='object']

Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')

In [20]:
train[train['Weight Capacity (kg)']=='Nan']

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
3168,Nan,Polyester,Large,1.0,No,No,Messenger,Nan,Nan,96.84190
4202,Nike,Leather,Nan,1.0,Yes,No,Nan,Black,Nan,20.40842
6314,Nan,Polyester,Large,8.0,No,Yes,Backpack,Red,Nan,149.68126
7102,Nan,Polyester,Medium,2.0,Nan,No,Tote,Pink,Nan,131.45734
10006,Adidas,Nan,Large,10.0,Yes,No,Nan,Red,Nan,113.26874
...,...,...,...,...,...,...,...,...,...,...
296132,Nan,Nylon,Large,10.0,Nan,Yes,Backpack,Gray,Nan,108.38381
297801,Adidas,Nylon,Small,1.0,Yes,Nan,Backpack,Red,Nan,76.21084
299000,Nan,Leather,Nan,9.0,Nan,No,Tote,Pink,Nan,86.49314
299642,Under Armour,Polyester,Medium,7.0,Yes,Nan,Nan,Black,Nan,27.85573


In [16]:
train['Brand'].unique()

array(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma', 'Nan'],
      dtype=object)

In [14]:
train = train.replace(np.nan, "Nan")


In [21]:
train['Weight Capacity (kg)']=train['Weight Capacity (kg)'].replace("Nan",np.nan)

<ipython-input-21-24ea5c55102d>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Weight Capacity (kg)']=train['Weight Capacity (kg)'].replace("Nan",np.nan)


In [5]:
import catboost
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [6]:
X=train.drop(['Price'],axis=1)
y=train['Price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [22]:
# Initialize CatBoost Regressor
catboost_model = CatBoostRegressor(
    iterations=500,  # Number of boosting iterations
    learning_rate=0.1,  # Step size shrinkage
    depth=6,  # Maximum depth of trees
    loss_function='RMSE',  # Root Mean Squared Error for regression
    verbose=100,
    cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color']# Print training progress every 100 iterations
)

# Train the model
catboost_model.fit(X_train, y_train)


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.